# Daten vorbereiten

### Bibliothek importieren

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from glob import glob 
import ipdb
import os
import os.path
import re
from tqdm.notebook import tqdm
import codecs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC
import scipy as sp
import pickle

### GUID file und Test-/Traindatei kombinieren

In [2]:
filelist = glob(r'C:\Users\lucab\male-biel\Stadt sg\extracts\*')
nlp = spacy.load('de_core_news_md')


In [3]:
# GUID CSV Traindaten
df1 = pd.read_csv(r'Stadt sg\traktandierte-geschaefte-sitzungen-stadtparlament-stgallen.csv',sep=';')

# GUID CSV Testdaten: hier = Traindaten
df2 = pd.read_csv(r'Stadt sg\traktandierte-geschaefte-sitzungen-stadtparlament-stgallen.csv',sep=';')

In [4]:
df2

,nr,Sitzungs_ID,Link zur Sitzung,Legislatur,Jahr,Bezeichnung,Sitzungsdatum,Freigabe,Ort,Zeit,...,Aktenplan Ebene5,Aktenplan Ebene6,Dokumentendatum,Dokumententitel,Traktandentitel,Vorberatende Kommission,Traktandenstatus,Traktandennummer,Geschaeft_GUID,Traktandum_GUID
0,2014-05-06T00:00:00.000-9,00731bb5b58e4aa0a731370c10e23550,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2013 - 31.12.2016,2014,16. Sitzung Stadtparlament,2014-05-06,Öffentlich,Waaghaus,16:00,...,Stadtparlament: Interpellationen,NaN,2014-05-13,Protokoll Stadtparlament 6. Mai 2014,"Interpellation Maria Pappa, Lisa Etter, Evelyn...",DIF,behandelt,9.0,272d778c03994b0f8dbded062204f36c,b92f0d8517024190a95109b1aa27d461
1,2010-02-16T00:00:00.000-1,02ba50062d9b43a5ab8efd3c7462bd0b,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2009 - 31.12.2012,2010,15. Sitzung Stadtparlament,2010-02-16,Öffentlich,NaN,16:00,...,NaN,NaN,2012-08-28,Protokoll StParl 16.02.2010,"Sporthalle Kreuzbleiche, dringende Gesamterneu...",Bauko,behandelt,1.0,abef5b68ab754b948eb25db1bf97f304,e113cf6f833546289c0663cf1daaec56
2,2010-02-16T00:00:00.000-2,02ba50062d9b43a5ab8efd3c7462bd0b,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2009 - 31.12.2012,2010,15. Sitzung Stadtparlament,2010-02-16,Öffentlich,NaN,16:00,...,NaN,NaN,2012-08-28,Protokoll StParl 16.02.2010,"Neubau eines Werkhofes für das Gartenbauamt, P...",Bauko,behandelt,2.0,039724f3b51e4937a91f92bd292ea1dd,a37de1d040c64152b3b5e16e5142d765
3,2010-02-16T00:00:00.000-10,02ba50062d9b43a5ab8efd3c7462bd0b,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2009 - 31.12.2012,2010,15. Sitzung Stadtparlament,2010-02-16,Öffentlich,NaN,16:00,...,Stadtparlament: Einfache Anfragen,NaN,2012-08-28,Protokoll StParl 16.02.2010,Jennifer Deuel-Zumstein: Welche Aufgaben nimmt...,NaN,behandelt,10.0,d9ff44a359ef426a82086a1683f47c35,b2a702ba8b8f48d89e9e118842016f1c
4,2006-11-28T00:00:00.000-4,02fa0fc0d010458db69ae0b3bfa0fc23,NaN,01.01.2005 - 31.12.2008,2006,30. Sitzung Stadtparlament,2006-11-28,Öffentlich,Waaghaus,NaN,...,NaN,NaN,2006-11-28,Protokoll StParl 28.11.2006,Zylistrasse - Güterbahnhofstrasse - Oberstrass...,NaN,behandelt,4.0,0b8d167752174e3bb14dbd4aeccaa82d,80798abd63e44df0a8858d635c5d9f5a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840,2022-03-22T00:00:00.000-20,66253656a9a947e49d19a5fdf30fa0ce,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2021 - 31.12.2024,2022,15. Sitzung Stadtparlament,2022-03-22,Öffentlich,Waaghaus,16:00,...,Stadtparlament: Interpellationen,NaN,NaN,NaN,"Interpellation SP/JUSO/PFG, Grüne/Junge Grüne,...",NaN,behandelt,20.0,1f93c31e836a4c5a8a6e4d963ffbc780,3e088105cf194e2d9254f671b3e615c6
3841,2022-03-22T00:00:00.000-25,66253656a9a947e49d19a5fdf30fa0ce,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2021 - 31.12.2024,2022,15. Sitzung Stadtparlament,2022-03-22,Öffentlich,Waaghaus,16:00,...,Stadtparlament: Einfache Anfragen,NaN,NaN,NaN,Einfache Anfrage Jacqueline Gasser-Beck: Hacka...,NaN,behandelt,25.0,c4ed6c69e60448feadeb0ea069dc9372,943016965e90475bb6b855800cc2b331
3842,2022-07-05T00:00:00.000-2,89af8518b34241fd98ff9a6873cb004b,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2021 - 31.12.2024,2022,Sitzung Stadtparlament Rechnung 2021,2022-07-05,Intern,Waaghaus,16:00,...,NaN,NaN,NaN,NaN,Geschäftsberichte der Stadt St.Gallen 2021,NaN,NaN,2.0,NaN,d35846b629e343bc9838e89ac4c27e74
3843,2022-05-03T00:00:00.000-3,6569ceead51f4d5183113ef281213276,https://www.stadt.sg.ch/home/verwaltung-politi...,01.01.2021 - 31.12.2024,2022,Sitzung Stadtparlament,2022-05-03,Intern,Waaghaus,16:00,...,NaN,NaN,NaN,NaN,Totalrevision des Reglements über die Finanzie...,LBK,NaN,3.0,6f7b765380144a298e7ea9eeb6fb6859,ebab0fb33be54fe2af284c25324415f0


In [5]:
dftrain = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\train.csv")

In [6]:
y = dftrain["y"]

In [7]:
for index,titel in dftrain.X1.iteritems():
    isinLA = df1.Traktandentitel.dropna().map(lambda s:titel.replace(')','').replace('-','') in s.replace(')','').replace('-',''))
    if not isinLA.any():
        print(f"{titel}",'nicht gefunden')
    else:
        Geschaeft_GUID = df1.loc[df1.Traktandentitel.dropna()[isinLA].index].Geschaeft_GUID
        Geschaeft_GUID = Geschaeft_GUID.unique().tolist()
        dftrain.loc[index,'Geschaeft_GUID']=str(Geschaeft_GUID)

Erteilung des Gemeinde- und Ortsbürgerrecht a) Einbürgerungsrat St.Gallen b) Einbürgerungsrat St.Gallen-Tablat c) Einbürgerungsrat St.Gallen-Straubenzell nicht gefunden
a) Wahl einer Ombudsperson b) Wahl einer Ombudsperson-Stellvertretung c) Ombudsstelle; Nachtragskredit nicht gefunden
Ersatz Reservoir und Pumpwerk Speicherstrasse; Ablauf Referendumsfrist nicht gefunden
Anpassung des Gebührenrahmens für die Besucher-Parkbewilligung; Nachtrag II zum Reglement über das Parkieren auf öffentlichem Grund (Parkierreglement) vom 28. November 2006 (sRS 712.2); Ablauf Referendumsfrist nicht gefunden
St.Gallen – Rorschacher Strasse; Erneuerung der Wassertransport- und der Wasserverteilleitung in der Rorschacher Strasse, Abschnitt Grütlistrasse bis Haus Nr. 250; Verpflichtungskredit; Ablauf Referendumsfrist nicht gefunden
Erteilung des Gemeinde- und Ortsbürgerrecht a) Einbürgerungsrat St.Gallen b) Einbürgerungsrat St.Gallen-Tablat c) Einbürgerungsrat St.Gallen-Straubenzell nicht gefunden
Erteilun

In [8]:
directory = 'C:\\Users\\lucab\\male-biel\\Stadt sg\\extracts\\'
ifound=0
inotfound=0
for index,fn in dftrain['Geschaeft_GUID'].dropna().items():
    l = eval(fn)
    found_any=False
    for g_GUID in l:
        path=os.path.join(r'C:\Users\lucab\male-biel\Stadt sg\extracts',g_GUID)
        if os.path.exists(path):
            found_any=True
    if found_any:
        ifound+=1
        dftrain.loc[index,'path']=path
    else:
        inotfound+=1
        dftrain.loc[index,'path']= '%snew_direct_%s' % (directory,str(inotfound))

print(ifound,'found,',inotfound,'not found')

1192 found, 5 not found


In [9]:
newdf = dftrain[(dftrain.path.isna())]
for Id in newdf.Id:
    dftrain.path.iloc[Id] = 'C:\\Users\\lucab\\male-biel\\Stadt sg\\extracts\\ReplaceNAN%s' %Id

C:\Users\lucab\AppData\Local\Temp\ipykernel_20684\175307637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftrain.path.iloc[Id] = 'C:\\Users\\lucab\\male-biel\\Stadt sg\\extracts\\ReplaceNAN%s' %Id


In [10]:
for i in dftrain.Id:
    os.makedirs(dftrain.path[i], exist_ok=True);
    
    with open("%s/title220407" % dftrain.path[i], 'w', encoding='utf8') as output_file:
        output_file.write(dftrain.X1[i])

### Spacy auf alle Files anwenden

In [11]:
def extracttext(s):
    print(s)
    if s=="":
        return ""
    else:   
        files = glob(os.path.join(s,'*'))
        text={}
        i=0
        x=""
        for file in files:
                    with codecs.open(file, "r", "utf-8") as fh:
                        lines = fh.readlines()
                        text[i] = ' '.join(lines).replace('- ','')
                        text[i] = re.sub(r"\d+([']\d+)*",'<NUMBER>',text[i])
                        if len(text[i]) > 100000:
                            text[i]=text[i][:100000]
                    i = i+1
        for n in range(0,i):
            x = x+text[n]
        return x

In [12]:
def extractspacysvector(s):
        x = extracttext(s)
        doc = nlp(x)
        a= doc.vector
        return a

In [13]:
XTrain2 = dftrain.path.apply(extractspacysvector)

C:\Users\lucab\male-biel\Stadt sg\extracts\5c5d97cb5d504af49081116941868858
C:\Users\lucab\male-biel\Stadt sg\extracts\26df226758334cb5bd353a3ddd6fb565
C:\Users\lucab\male-biel\Stadt sg\extracts\81554756428a40b69f08b73849e97642
C:\Users\lucab\male-biel\Stadt sg\extracts\7fbf9d1b99f94cf0bc1f2a72778b2032
C:\Users\lucab\male-biel\Stadt sg\extracts\a7cf239d72ff4cf6bc5853e54df8492d
C:\Users\lucab\male-biel\Stadt sg\extracts\d0a3ee1502d641e582ae56bf651deef0
C:\Users\lucab\male-biel\Stadt sg\extracts\bde59684a486479e996a9b0a82e220d3
C:\Users\lucab\male-biel\Stadt sg\extracts\c3fff74b2ef741fab14f96d304815d43
C:\Users\lucab\male-biel\Stadt sg\extracts\a2a2f68d049d4f2a8b5a2ba7b5db606c
C:\Users\lucab\male-biel\Stadt sg\extracts\8e606f8b96d4407e94e7e237a2ba982c
C:\Users\lucab\male-biel\Stadt sg\extracts\8ff39e06082e42639f0cfbc6efa6a605
C:\Users\lucab\male-biel\Stadt sg\extracts\4a27f52eb0f34190b3d1701d3afd1dc8
C:\Users\lucab\male-biel\Stadt sg\extracts\10dbcfba9d994555837755d301611440
C:\Users\luc

## dftrain = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\train.csv")
dftest = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\test.csv")
sw = pd.read_csv(r"C:\Users\lucab\male-biel\Stopwords.csv",sep=";",header=None)
sw = sw[0].tolist()

### Tfidf auf alle Files anwenden

In [14]:
Train = dftrain.path.apply(extracttext)

C:\Users\lucab\male-biel\Stadt sg\extracts\5c5d97cb5d504af49081116941868858
C:\Users\lucab\male-biel\Stadt sg\extracts\26df226758334cb5bd353a3ddd6fb565
C:\Users\lucab\male-biel\Stadt sg\extracts\81554756428a40b69f08b73849e97642
C:\Users\lucab\male-biel\Stadt sg\extracts\7fbf9d1b99f94cf0bc1f2a72778b2032
C:\Users\lucab\male-biel\Stadt sg\extracts\a7cf239d72ff4cf6bc5853e54df8492d
C:\Users\lucab\male-biel\Stadt sg\extracts\d0a3ee1502d641e582ae56bf651deef0
C:\Users\lucab\male-biel\Stadt sg\extracts\bde59684a486479e996a9b0a82e220d3
C:\Users\lucab\male-biel\Stadt sg\extracts\c3fff74b2ef741fab14f96d304815d43
C:\Users\lucab\male-biel\Stadt sg\extracts\a2a2f68d049d4f2a8b5a2ba7b5db606c
C:\Users\lucab\male-biel\Stadt sg\extracts\8e606f8b96d4407e94e7e237a2ba982c
C:\Users\lucab\male-biel\Stadt sg\extracts\8ff39e06082e42639f0cfbc6efa6a605
C:\Users\lucab\male-biel\Stadt sg\extracts\4a27f52eb0f34190b3d1701d3afd1dc8
C:\Users\lucab\male-biel\Stadt sg\extracts\10dbcfba9d994555837755d301611440
C:\Users\luc

In [15]:
sw = pd.read_csv(r"C:\Users\lucab\male-biel\Stopwords.csv",sep=";",header=None)
sw = sw[0].tolist()

In [16]:
vct1 =  TfidfVectorizer(stop_words=sw) 
vct2 =  TfidfVectorizer(stop_words=sw) 

In [17]:
XTrain3 = vct1.fit_transform(Train)

C:\Users\lucab\anaconda3\envs\MALE\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wahr'] not in stop_words.
  warnings.warn(


### Anpassung X1 mit Spacy

In [18]:
def spacylemma(s):
    doc = nlp(s)
    a = ""
    for token in doc:
        a = a+ token.lemma_+" " 
    return a

In [19]:
dftrain['XS']=dftrain.X1.apply(spacylemma)

In [20]:
XTrain1 = vct2.fit_transform(dftrain.XS)

C:\Users\lucab\anaconda3\envs\MALE\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['wahr'] not in stop_words.
  warnings.warn(


XTrainsafe1 = pd.DataFrame(data=XTrain1.toarray())
XTestsafe1 = pd.DataFrame(data=XTest1.toarray())
XTrainsafe1.to_csv(path_or_buf=r"Stadt sg\trainx1.csv")
XTestsafe1.to_csv(path_or_buf=r"Stadt sg\testx1.csv")
XTrainsafe2 = pd.DataFrame(data=XTrain2)
XTestsafe2 = pd.DataFrame(data=XTest2)
XTrainsafe2.to_csv(path_or_buf=r"Stadt sg\trainx2.csv")
XTestsafe2.to_csv(path_or_buf=r"Stadt sg\testx2.csv")
XTrainsafe3 = pd.DataFrame(data=XTrain3.toarray())
XTestsafe3 = pd.DataFrame(data=XTest3.toarray())
XTrainsafe3.to_csv(path_or_buf=r"Stadt sg\trainx3.csv")
XTestsafe3.to_csv(path_or_buf=r"Stadt sg\testx3.csv")

In [21]:
y.to_csv(path_or_buf=r"Stadt sg\xy.csv", index=False)

### Prediction mittels SVM auf Titeln mit TFIDF

XTrain1 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\trainx1.csv")
XTest1 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\testx1.csv")

y = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\xy.csv")

In [22]:
y = dftrain.y

XTrain1 = sp.sparse.csr_matrix(np.array(XTrain1.values.tolist()))
XTest1 = sp.sparse.csr_matrix(np.array(XTest1.values.tolist()))

In [23]:
type(XTrain2)

pandas.core.series.Series

In [24]:
clf1=SVC(kernel='poly',probability=True,C=2,degree=1,gamma=1)

In [25]:
clf1.fit(XTrain1,y)
clf1.score(XTrain1,y)

0.9950166112956811

### Prediction mittels Random Forest auf allem mit Spacy

XTrain2 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\trainx2.csv")
XTest2 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\testx2.csv")

XTrain2 = np.array(XTrain2.values.tolist())
XTest2 = np.array(XTest2.values.tolist())

In [26]:
XTrain2 = np.array(XTrain2.values.tolist())

In [27]:
clf2=RandomForestClassifier(n_jobs=-1, n_estimators= 252, max_depth= 158, criterion= 'entropy')
clf2.fit(XTrain2,y)
clf2.score(XTrain2,y)


0.9983388704318937

### Prediction mittels SVM auf allem mit TFIDF

XTrain3 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\trainx3.csv")
XTest3 = pd.read_csv(r"C:\Users\lucab\male-biel\Stadt sg\testx3.csv")

XTrain3 = sp.sparse.csr_matrix(np.array(XTrain3.values.tolist()).todense())
XTest3 = sp.sparse.csr_matrix(np.array(XTest3.values.tolist()).todense())

In [28]:
clf3 = SVC(probability = True, C= 21, degree= 3, gamma= 0.8485565656565657, kernel= 'linear')

In [29]:
clf3.fit(XTrain3,y)
clf3.score(XTrain3,y)

0.9626245847176079

In [30]:
complexpickle = {'vct1':vct1,'vct2':vct2, 'clf1':clf1, 'clf2':clf2, 'clf3':clf3}

In [31]:
with open('Stadt sg\\complexpickle.pickle', 'wb') as f:
    pickle.dump(complexpickle, f)